LV-point-01.

Example of an optimization to minimize the distance between observed and predicted solutions of a Lotka-Volterra system to find point estimates of parameters and initial conditions for a system.


In [1]:
using DifferentialEquations, Flux, Optim, DiffEqFlux, DiffEqSensitivity, Plots
using Plotly
plotly()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5352085366034723864\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8590318625121000724\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5352085366034723864\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/airwin/.julia/packages/Plots/cc8wh/src/backends.jl:363


Plots.PlotlyBackend()


First, define and solve the LV system.

We are going to need several different solutions to this system:
sol: a high resolution solution to the original system
sol_discrete: a solution sampled at a few time points

For our first estimation we will write
p_est1: estimated parmeters for this optimization
sol_est1: a solution using these parameters

In [2]:
function lotka_volterra!(du, u, p, t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

# Initial condition
u0 = [1.0, 1.0]

# Simulation interval and intermediary points
tspan = (0.0, 5.0)
tsteps = 0.0:0.5:5.0

# LV equation parameter. p = [α, β, δ, γ]
p = [1.5, 1.0, 3.0, 1.0]

# Setup the ODE problem, then solve
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob, Tsit5());
sol_discrete = solve(prob, Tsit5(), saveat = tsteps);

Plot the solution and the discrete times to be used in the optimization.

In [3]:
Plots.plot(sol )
Plots.scatter!(sol_discrete) # vars = [(0,1), (0,2)]

<!DOCTYPE html>
 
 
 Plots.jl

Define an objective function to be minimized between the data and a solution. Call an optimizer to find parameter values. Use a callback function to observe the parameter values at each iteration of the optimization.

In [4]:
function loss(p)
  sol0 = solve(prob, Tsit5(), p=p, saveat = tsteps)
  loss = sum(sum.([v.^2 for v in (sol0.u - sol_discrete.u)]))

  return loss, sol0
end

callback = function (p, l, pred)
    display(p)
  # display(l)
  # plt = Plots.plot(pred, ylim = (0, 6))
  # display(plt)
  # Tell sciml_train to not halt the optimization. If return true, then
  # optimization stops.
  return false
end

p_est1 = DiffEqFlux.sciml_train(loss, p,
                                    ADAM(0.1),
                                    cb = callback,
                                    maxiters = 200)

4-element Array{Float64,1}:
 1.5
 1.0
 3.0
 1.0

4-element Array{Float64,1}:
 1.4999990993774355
 0.9999999478942987
 3.0000001043842324
 0.9999987643225617

4-element Array{Float64,1}:
 1.5744119608236116
 0.9255911154362209
 2.9256220690042385
 1.0744113292962711

4-element Array{Float64,1}:
 1.510531155153534
 0.9894721791258955
 2.8617400449337373
 1.0105306488056984

4-element Array{Float64,1}:
 1.4587677742215897
 1.041672370123607
 2.9047522006595283
 0.9437549583734569

...

4-element Array{Float64,1}:
 1.4999975236971619
 1.0000023537633946
 3.000008676619271
 1.0000086429192214

4-element Array{Float64,1}:
 1.4999953872722713
 1.0000050341877742
 3.0000080356017014
 1.0000071202913334

4-element Array{Float64,1}:
 1.4999950243018003
 1.0000049370037234
 3.0000112482057815
 1.0000045050701447

4-element Array{Float64,1}:
 1.4999966806521705
 1.0000022239645916
 3.000016043487006
 1.0000026173927585

u: 4-element Array{Float64,1}:
 1.4999966806521705
 1.0000022239645916
 3.000016043487006
 1.0000026173927585

Compute and plot the smooth solution for these parameters. Show discrete data used for optimization fitting as well.

In [5]:
sol_est1 = solve(remake(prob, p = p_est1.u), Tsit5());

In [6]:
Plots.scatter(sol_discrete)
Plots.plot!(sol_est1)

<!DOCTYPE html>
 
 
 Plots.jl

Add some measurement error to the data and estimate the initial conditions as well.

In [7]:
data1 = Array(sol_discrete) .+ 0.8 * randn(size(Array(sol_discrete)));
Plots.plot(sol)
Plots.scatter!(sol_discrete.t, data1')

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
function loss(p)
    sol0 = solve(prob, Tsit5(), u0 = p[1:2], p=p[3:end], saveat = tsteps)
  # loss = sum(sum.([v.^2 for v in (sol0.u - sol.u)]))
    loss = sum((data1 - Array(sol0)).^2)

  return loss, sol0
end

callback = function (p, l, pred)
    display(p)
  # display(l)
  # plt = Plots.plot(pred, ylim = (0, 6))
  # display(plt)
  # Tell sciml_train to not halt the optimization. If return true, then
  # optimization stops.
  return false
end
u0p = [u0; p]
p_est2 = DiffEqFlux.sciml_train(loss, u0p,
                                    ADAM(0.1),
                                    cb = callback,
                                    maxiters = 200)


6-element Array{Float64,1}:
 1.0
 1.0
 1.5
 1.0
 3.0
 1.0

6-element Array{Float64,1}:
 1.0999999998540582
 1.0999999998791283
 1.5999999998915049
 1.0999999999144412
 2.9000000009018985
 1.0999999999340957

6-element Array{Float64,1}:
 1.0312449477417638
 1.1869739047912624
 1.5299961734452996
 1.1907783086275607
 2.8146411355684684
 1.0367724876119957

6-element Array{Float64,1}:
 0.980538404003718
 1.2431060329608825
 1.4999247439736443
 1.2397401129146073
 2.8605075596425595
 0.9739497943503338

6-element Array{Float64,1}:
 0.974674818746255
 1.2470044523213069
 1.5194286547904747
 1.2363475760267288
 2.928201624502942
 0.9423415251514804

...

6-element Array{Float64,1}:
 0.8269640728109171
 2.046387857016042
 2.3192431973013052
 1.5065002326128227
 2.184959257355561
 0.7769751610734218

6-element Array{Float64,1}:
 0.8266746307556179
 2.0463183085845364
 2.320832096835042
 1.5072458531924253
 2.1832861359294777
 0.776454984288985

6-element Array{Float64,1}:
 0.8263823200762195
 2.046263785739516
 2.3223917693062637
 1.508006986687624
 2.1816175979688097
 0.7759309865255075

6-element Array{Float64,1}:
 0.8260744680194422
 2.0462447350138055
 2.3239077878398664
 1.5088095727468525
 2.1799114208720813
 0.7754001348944143

u: 6-element Array{Float64,1}:
 0.8260744680194422
 2.0462447350138055
 2.3239077878398664
 1.5088095727468525
 2.1799114208720813
 0.7754001348944143

In [9]:
sol_est2 = solve(remake(prob, u0 = p_est2.u[1:2], p = p_est2.u[3:end]), Tsit5());

In [10]:
Plots.scatter(sol_discrete.t, data1')  # data used to estimate parameters and initial conditions
Plots.plot!(sol_est2)   # solution from noisy data shown with thin lines
Plots.plot!(sol, lw=2)  # solution to original system with thick lines

<!DOCTYPE html>
 
 
 Plots.jl

Also look at numerical values of initial conditions u0 and parameters p


In [11]:
[u0' p'   # original problem
    u0' p_est1'  # estimated from perfect data
p_est2'] # estimated from noisy data

3×6 Array{Float64,2}:
 1.0       1.0      1.5      1.0      3.0      1.0
 1.0       1.0      1.5      1.0      3.00002  1.0
 0.826074  2.04624  2.32391  1.50881  2.17991  0.7754